In [69]:
from neo4j import GraphDatabase
import redis
import json
from dotenv import load_dotenv
import os

load_dotenv()

True

In [70]:
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
URI = os.getenv('NEO4J_URI')

In [71]:
# Neo4j connection
uri = URI
username = "neo4j"
password = NEO4J_PASSWORD
AUTH = (username, password)

driver = GraphDatabase.driver(uri, auth=(username, password))

# Create a session
session = driver.session()

# Redis connection
r = redis.StrictRedis(host='localhost', port=6379, db=0)

In [72]:
# Function to get all players from Neo4j
def get_all_players_from_neo4j():
    with session as s:
        result = s.run("MATCH (p:PLAYER) RETURN p.name AS name, p.age AS age, p.number AS number")
        players = []
        for record in result:
            player_data = {
                'name': record['name'],
                'age': record['age'],
                'number': record['number']
            }
            players.append(player_data)
    return players


In [ ]:
def cache_players_in_redis():
    players = get_all_players_from_neo4j()
    for player in players:

        player_id = player['name']

        player_data_json = json.dumps(player)

        r.set(f"player:{player_id}", player_data_json)
        
        # Optionally, set an expiration time for the cache (e.g., 3600 seconds = 1 hour)
        r.expire(f"player:{player_id}", 3600)
    print(f"Successfully cached {len(players)} players in Redis.")

# Call the function to cache players
cache_players_in_redis()

Successfully cached 12 players in Redis.


In [74]:
def get_player_from_cache(player_id):
    cached_data = r.get(f"player:{player_id}")
    if cached_data:

        return json.loads(cached_data)
    else:

        return None

In [75]:
# Example of checking the cache for a specific player
player_id = 'LeBron James'  # Example player ID
# Russell Westbrook

In [76]:
player_data = get_player_from_cache(player_id)
if player_data:
    print(f"Player data retrieved from cache: {player_data}")
else:
    print("Player data not found in cache. Querying Neo4j...")

Player data retrieved from cache: {'name': 'LeBron James', 'age': 36, 'number': 6}
